<a href="https://colab.research.google.com/github/kr7/timeseriesforecast-siconv/blob/main/TimeSeriesForecast_SiConv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import random
import scipy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from numpy import genfromtxt
from sklearn.model_selection import KFold

# Parameters

In [ ]:
FORECAST_HORIZON = 16
SPARSITY = 0.8 # SPARSITY=0.8 means that 80% of the values of the time series will be replaced by zeros (i.e., missing values)

## Load the data

In [ ]:
!wget http://www.timeseriesclassification.com/Downloads/Archives/Univariate2018_arff.zip

In [ ]:
!unzip Univariate2018_arff.zip

In [ ]:
# Change here to run experiments on other datasets
file_name_prefix = "Univariate_arff/Adiac/Adiac"

In [ ]:
def sparsify_time_series(time_series, p):
  number_of_missing = int(p*len(time_series))
  for i in range(number_of_missing):
    j = random.randint(0,len(time_series)-1)
    while time_series[j] == 0:
      j = random.randint(0,len(time_series)-1)
    time_series[j] = 0
  return time_series

def sparsify_time_series_dataset(ts_data, p):
  for i in range(len(ts_data)):
    ts_data[i] = sparsify_time_series(ts_data[i], p)
  return ts_data

In [ ]:
# In order to perform 10-fold cross-validation, we
# will merge the provided train and test splits and 
# we will split the data durign the cross-validation

train_data_with_class_labels = np.genfromtxt(file_name_prefix+'_TRAIN.txt')
test_data_with_class_labels = np.genfromtxt(file_name_prefix+'_TEST.txt')

data_with_class_labels = np.vstack( (train_data_with_class_labels, 
                                     test_data_with_class_labels))
data_without_class_labels = data_with_class_labels[:,1:]
data_without_class_labels = sparsify_time_series_dataset(data_without_class_labels, SPARSITY)
input_data = data_without_class_labels[:,:-FORECAST_HORIZON]
target = data_without_class_labels[:,-FORECAST_HORIZON:]

# We make sure that the length of the time series is a multiple of 4 

NUM_INPUT_FEATURES = len(input_data[0]) 
values_to_cut = NUM_INPUT_FEATURES % 4
if values_to_cut != 0:
    input_data = input_data[:,values_to_cut:]
    NUM_INPUT_FEATURES = NUM_INPUT_FEATURES - values_to_cut

## Sparsity-invariant convolution and sparsity-aware MSE loss

In [ ]:
class SparseConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(SparseConvolution, self).__init__()
        self.stride = stride
        self.padding = padding
        self.conv = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, 
                              kernel_size=kernel_size, stride=stride, padding=padding)
        self.ones = torch.ones(out_channels, in_channels, kernel_size).cuda()

    def forward(self, x):
      non_zero_input = (x!=0).float()
      number_of_non_zero_inputs = F.conv1d(non_zero_input, self.ones, stride=self.stride, padding=self.padding)
      mask = (number_of_non_zero_inputs!=0)
      
      x = self.conv(x)
      x[mask] = x[mask]/number_of_non_zero_inputs[mask]
      return x

class SparseMSELoss(nn.Module):
    def __init__(self):
        super(SparseMSELoss, self).__init__()

    def forward(self, pred, target):
        epsilon = 1e-5
        mask = (target != 0)
        return torch.sum(((mask*pred)-target)**2)/(torch.sum(mask)+epsilon)

# The model

In [ ]:
# Definition of the neural networks

CONV_FILTERS = 25
CONV_FILTER_SIZE = 9


class CNN(nn.Module):
    def __init__(self, convolution_type='conventional'):
        super(CNN, self).__init__()
        num_units_fc = 100
        self.num_inputs_fc = int(CONV_FILTERS*(NUM_INPUT_FEATURES-CONV_FILTER_SIZE+1)/2)

        if convolution_type == 'conventional':
          self.conv1 = nn.Conv1d(in_channels = 1, out_channels = CONV_FILTERS, 
                               kernel_size=CONV_FILTER_SIZE, padding = 0, stride = 1)
        elif convolution_type == 'sparse':
          self.conv1 = SparseConvolution(in_channels = 1, out_channels = CONV_FILTERS, 
                               kernel_size=CONV_FILTER_SIZE, padding = 0, stride = 1)
        self.max_pool = nn.MaxPool1d(2)
        self.fc = nn.Linear(self.num_inputs_fc, num_units_fc)
        self.out = nn.Linear(num_units_fc, FORECAST_HORIZON) 

    def forward(self, x):
        x = x.view(-1, 1, NUM_INPUT_FEATURES)
        x = self.conv1(x)
        x = self.max_pool(x)
        x = x.view(-1, self.num_inputs_fc)
        x = torch.relu(self.fc(x))
        x = self.out(x)
        return x

# The function used to evaluate the network 

In [ ]:
def eval_net(net, test_input, test_target):
    test_dataset = torch.utils.data.TensorDataset( 
        torch.Tensor(test_input), 
        torch.Tensor(test_target)
    )
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=1)

    mae = 0
    mse = 0
    total = 0

    with torch.no_grad():
        for inputs, true_target in testloader:
            inputs = inputs.cuda()
            true_target = true_target.cuda()
            
            predicted_target = net(inputs).cpu().numpy()
            true_target = true_target.cpu().numpy()
            
            mask = (true_target!=0)
            predicted_target = mask*predicted_target

            mae += np.sum(np.abs(predicted_target - true_target))
            mse += np.sum((predicted_target - true_target)**2)
            total += np.sum(mask)

    return mse/total, mae/total

# Function for linear interpolation of sparse time series 

In [ ]:
def linear_interpolation(ts):
  previous_value = 0
  previous_index = -1
  for i in range(len(ts)):
    if ts[i] != 0:
      previous_index = i
      previous_value = ts[previous_index]
    else:
      next_value = 0
      next_index = i + 1
      while next_index < len(ts) and ts[next_index] == 0:
        next_index = next_index+1
      if next_index < len(ts):
        next_value = ts[next_index]

      w_next = (i-previous_index) / (next_index - previous_index)
      w_previous = 1-w_next

      ts[i] = w_next*next_value + w_previous*previous_value
  return ts

def linear_interpolation_all(ts_data):
  interpolated_data = np.array(ts_data)
  for i in range(len(interpolated_data)):
    interpolated_data[i] = linear_interpolation(interpolated_data[i])
  return interpolated_data

# The main experimental loop

In [ ]:
epochs = 1000

mse_cnn  = []
mse_cnn_sparse = []
mse_cnn_linear = []
mae_cnn  = []
mae_cnn_sparse = []
mae_cnn_linear = []

kf = KFold(n_splits=10, random_state=42, shuffle=True)

fold = 0
for train_index, test_index in kf.split(input_data):
    fold = fold + 1

    train_data = input_data[train_index]
    train_target = target[train_index]
    test_data = input_data[test_index]
    test_target = target[test_index]

    # Training of CNN. This is simultaneously the initial stage of training of the DCNN.

    train_dataset = torch.utils.data.TensorDataset(
      torch.Tensor(train_data), 
      torch.Tensor(train_target) 
    )
    trainloader = torch.utils.data.DataLoader(
      train_dataset, shuffle=True, batch_size=16)

    
    # Train CNN with sparsity-invariant convolution

    cnn_sparse = CNN("sparse")
    cnn_sparse.cuda()
    criterion = SparseMSELoss()
    optimizer = optim.Adam(cnn_sparse.parameters(), lr=1e-5)

    running_loss = 0.0
    running_n = 0

    print("Training sparse CNN...")

    for epoch in range(epochs):  
        for input_batch, target_batch in trainloader:
            input_batch = input_batch.cuda()
            target_batch = target_batch.cuda()

            optimizer.zero_grad()

            prediction_batch = cnn_sparse(input_batch) 

            loss = criterion(prediction_batch, target_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_n = running_n + 1

        if epoch % 100 == 0:
            print("epoch: {:3d} loss: {:4.3f}".format(epoch, running_loss/running_n))
            running_loss = 0.0
            running_n = 0

    # Train baseline CNN
    cnn = CNN()
    cnn.cuda()
    criterion = SparseMSELoss()
    optimizer = optim.Adam(cnn.parameters(), lr=1e-5)

    running_loss = 0.0
    running_n = 0

    print("Training CNN...")

    for epoch in range(epochs):  
        for input_batch, target_batch in trainloader:
            input_batch = input_batch.cuda()
            target_batch = target_batch.cuda()

            optimizer.zero_grad()

            prediction_batch = cnn(input_batch)

            loss = criterion(prediction_batch, target_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_n = running_n + 1

        if epoch % 100 == 0:
            print("epoch: {:3d} loss: {:4.3f}".format(epoch, running_loss/running_n))
            running_loss = 0.0
            running_n = 0

    # Train baseline CNN with linear interpolation

    train_dataset = torch.utils.data.TensorDataset(
      torch.Tensor(linear_interpolation_all(train_data)), 
      torch.Tensor(linear_interpolation_all(train_target)) 
    )
    trainloader = torch.utils.data.DataLoader(
      train_dataset, shuffle=True, batch_size=16)

    cnn_linear = CNN()
    cnn_linear.cuda()
    criterion = SparseMSELoss()
    optimizer = optim.Adam(cnn_linear.parameters(), lr=1e-5)

    running_loss = 0.0
    running_n = 0

    print("Training CNN with linear interpolation...")

    for epoch in range(epochs):  
        for input_batch, target_batch in trainloader:
            input_batch = input_batch.cuda()
            target_batch = target_batch.cuda()

            optimizer.zero_grad()

            prediction_batch = cnn_linear(input_batch)

            loss = criterion(prediction_batch, target_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_n = running_n + 1

        if epoch % 100 == 0:
            print("epoch: {:3d} loss: {:4.3f}".format(epoch, running_loss/running_n))
            running_loss = 0.0
            running_n = 0

    a_mse, a_mae = eval_net(cnn, test_data, test_target)
    mse_cnn.append(a_mse)
    mae_cnn.append(a_mae)

    a_mse_linear, a_mae_linear = eval_net(cnn_linear, test_data, test_target)
    mse_cnn_linear.append(a_mse_linear)
    mae_cnn_linear.append(a_mae_linear)

    a_mse_sparse, a_mae_sparse = eval_net(cnn_sparse, test_data, test_target)
    mse_cnn_sparse.append(a_mse_sparse)
    mae_cnn_sparse.append(a_mae_sparse)

    print(f"Fold: {fold:2d}")
    print(f"  MSE of CNN:    {a_mse:6.4f}")
    print(f"  MSE of linCNN: {a_mse_linear:6.4f}")
    print(f"  MSE of siCNN:  {a_mse_sparse:6.4f}")
    print(f"  MAE of CNN:    {a_mae:6.4f}")
    print(f"  MAE of linCNN: {a_mae_linear:6.4f}")
    print(f"  MAE of siCNN:  {a_mae_sparse:6.4f}")

# Print results, calculate p-values

In [ ]:
print(file_name_prefix.split('/')[-1])
print(f"Mean MSE CNN:    {np.mean(mse_cnn):6.4f}")
print(f"Mean MSE linCNN: {np.mean(mse_cnn_linear):6.4f}")
print(f"Mean MSE siCNN:  {np.mean(mse_cnn_sparse):6.4f}")
print(f"Std. MSE CNN:    {np.std(mse_cnn):6.4f}")
print(f"Std. MSE linCNN: {np.std(mse_cnn_linear):6.4f}")
print(f"Std. MSE siCNN:  {np.std(mse_cnn_sparse):6.4f}")
print(f"p-value:         {scipy.stats.ttest_rel(mse_cnn, mse_cnn_sparse)[1]:6.4f}")
print(f"p-value:         {scipy.stats.ttest_rel(mse_cnn_linear, mse_cnn_sparse)[1]:6.4f}")

print(f"Mean MAE CNN:    {np.mean(mae_cnn):6.4f}")
print(f"Mean MAE linCNN: {np.mean(mae_cnn_linear):6.4f}")
print(f"Mean MAE siCNN:  {np.mean(mae_cnn_sparse):6.4f}")
print(f"Std. MAE CNN:    {np.std(mae_cnn):6.4f}")
print(f"Std. MAE linCNN: {np.std(mae_cnn_linear):6.4f}")
print(f"Std. MAE siCNN:  {np.std(mae_cnn_sparse):6.4f}")
print(f"p-value:         {scipy.stats.ttest_rel(mae_cnn, mae_cnn_sparse)[1]:6.4f}")
print(f"p-value:         {scipy.stats.ttest_rel(mae_cnn_linear, mae_cnn_sparse)[1]:6.4f}")